In [23]:
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [24]:
df_order_historic_demand = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_historic_order_demand.xlsx')
df_distance_km = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_distance_km.xlsx')
df_distance_min = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_distance_min.xlsx')
df_location = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_location.xlsx')
df_orders = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_orders.xlsx')
df_vehicle = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_vehicle.xlsx')


In [25]:

ciudad = "Madrid, Spain"

G = ox.graph_from_place(ciudad, network_type="drive")

puntosTotales = []

df_location["Nodo"] = df_location.apply(
    lambda row: ox.distance.nearest_nodes(G, X=row["Longitud"], Y=row["Latitud"]), axis=1
)
recorrido = pd.DataFrame

#PRUEBAS
df_distance_min.index = df_distance_min.columns

# Convertir el DataFrame a una matriz NumPy
distance_matrix = df_distance_min.values
labels = df_distance_min.index.tolist()  # Obtener los nombres reales de filas/columnas

# Verificar etiquetas
print("Etiquetas en el índice:", labels)


Etiquetas en el índice: ['Cliente_1', 'Cliente_2', 'Cliente_3', 'Cliente_4', 'Cliente_5', 'Cliente_6', 'Cliente_7', 'Cliente_8', 'Cliente_9', 'Cliente_10', 'Cliente_11', 'Cliente_12', 'Cliente_13', 'Cliente_14', 'Cliente_15', 'Cliente_16', 'Cliente_17', 'Cliente_18', 'Cliente_19', 'Cliente_20', 'Almacén']


In [26]:
# Buscar "Almacén" con coincidencia exacta
almacen_index = labels.index("Almacén")  # Buscar el índice de "Almacén" en las etiquetas

# Implementación del algoritmo Nearest Neighbor
def nearest_neighbor_tsp(matrix, start):
    n = matrix.shape[0]
    visited = [start]  # Iniciar desde el almacén
    total_distance = 0

    while len(visited) < n:
        current = visited[-1]
        # Buscar la distancia mínima a un nodo no visitado
        distances = matrix[current, :]
        distances[visited] = np.inf  # Ignorar los ya visitados
        next_node = np.argmin(distances)
        total_distance += distances[next_node]
        visited.append(next_node)

    # Volver al almacén ? Revisar mas tarde
    total_distance += matrix[visited[-1], start]
    visited.append(start)
    return visited, total_distance

route, total_distance = nearest_neighbor_tsp(distance_matrix, almacen_index)

# Convertir índices de la ruta a nombres de clientes
route_labels = [labels[i] for i in route]

route_clients = [client for client in route_labels if client != "Almacén"]

print(route_clients)

['Cliente_9', 'Cliente_5', 'Cliente_3', 'Cliente_6', 'Cliente_4', 'Cliente_16', 'Cliente_10', 'Cliente_14', 'Cliente_7', 'Cliente_13', 'Cliente_2', 'Cliente_19', 'Cliente_12', 'Cliente_11', 'Cliente_1', 'Cliente_18', 'Cliente_15', 'Cliente_8', 'Cliente_17', 'Cliente_20']


In [27]:
#PARTE 2

df_location = df_location.set_index('Cliente').reindex(route_clients).reset_index()



In [28]:
coordinates = df_location[['Latitud', 'Longitud']].values

from sklearn.cluster import DBSCAN


# Configuración de rangos objetivo
rangos_clusters = {
    "1": range(1, 2),
    "1 a 2": range(1, 3),
    "1 a 3": range(1, 4),
    "1 a 4": range(1, 5),
}

# Hashmap para guardar todos los resultados
hashmap_rangos = {rango: {} for rango in rangos_clusters.keys()}

# Función principal
def encontrar_clusters(rango_nombre, rango_objetivo):
    eps = 0.01  # Valor inicial de eps
    min_samples = 2  # Número mínimo de puntos para formar un clúster
    iteration = 1  # Contador de iteraciones
    clientes_totales = set(df_location["Cliente"])  # Conjunto de todos los clientes
    
    while True:
        # Aplicar DBSCAN
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(coordinates)
        
        # Manejo de puntos ruido
        if -1 in labels:
            ruido_indices = np.where(labels == -1)[0]
            clúster_indices = np.where(labels != -1)[0]
            if len(clúster_indices) > 0:
                vecinos = NearestNeighbors(n_neighbors=1).fit(coordinates[clúster_indices])
                _, indices = vecinos.kneighbors(coordinates[ruido_indices])
                labels[ruido_indices] = labels[clúster_indices[indices.flatten()]]
        
        # Contar clústeres únicos
        num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        # Verificar si el número de clústeres está dentro del rango objetivo
        if num_clusters in rango_objetivo:
            # Crear hashmap para los grupos
            hashmap_grupos = {}
            for cluster_id in set(labels):
                if cluster_id == -1:
                    continue  # Ignorar puntos ruido
                miembros = df_location[np.array(labels) == cluster_id]["Cliente"].tolist()
                hashmap_grupos[cluster_id] = miembros
            
            # Verificar integridad: Todos los clientes deben estar asignados
            clientes_asignados = set(
                cliente for miembros in hashmap_grupos.values() for cliente in miembros
            )
            if clientes_totales != clientes_asignados:
                print(
                    f"Advertencia: Faltan clientes en rango {rango_nombre}, iteración {iteration}"
                )
                eps += 0.01  # Incrementar eps y continuar
                continue
            
            # Validar que no se dupliquen resultados para el mismo num_clusters
            ya_registrado = any(
                resultado["num_clusters"] == num_clusters
                for resultado in hashmap_rangos[rango_nombre].values()
            )
            if not ya_registrado:
                hashmap_rangos[rango_nombre][f"Iteración_{iteration}"] = {
                    "eps": eps,
                    "num_clusters": num_clusters,
                    "clusters": hashmap_grupos,
                }
            
            # Si alcanzamos el valor máximo del rango objetivo, detener
            if num_clusters == max(rango_objetivo):
                break
        
        # Incrementar eps dinámicamente y manejar iteraciones
        eps += 0.01
        iteration += 1
        if eps > 0.5:
            eps = 0.01
            min_samples += 1
        
        # Evitar bucles infinitos
        if iteration > 100:
            print(f"No se encontró un número de clústeres válido para el rango {rango_nombre}")
            break


# Simulación de coordenadas y datos
coordinates = np.random.rand(20, 2)  # Coordenadas ficticias
clientes = [f"Cliente_{i}" for i in range(1, 21)]
df_location = pd.DataFrame({"Cliente": clientes, "x": coordinates[:, 0], "y": coordinates[:, 1]})

# Ejecutar para cada rango de clústeres
for rango, objetivo in rangos_clusters.items():
    encontrar_clusters(rango, objetivo)

print(hashmap_rangos)

# Mostrar resultados finales
print("\n### Resultados Finales ###")
for rango, resultados in hashmap_rangos.items():
    print(f"\nRango {rango}:")
    for iteracion, datos in resultados.items():
        print(f"  {iteracion}: eps={datos['eps']}, num_clusters={datos['num_clusters']}")
        for cluster_id, miembros in datos["clusters"].items():
            print(f"    Cluster {cluster_id}: {', '.join(miembros)}")

hashmap_rangos

{'1': {'Iteración_48': {'eps': 0.48000000000000026, 'num_clusters': 1, 'clusters': {0: ['Cliente_1', 'Cliente_2', 'Cliente_3', 'Cliente_4', 'Cliente_5', 'Cliente_6', 'Cliente_7', 'Cliente_8', 'Cliente_9', 'Cliente_10', 'Cliente_11', 'Cliente_12', 'Cliente_13', 'Cliente_14', 'Cliente_15', 'Cliente_16', 'Cliente_17', 'Cliente_18', 'Cliente_19', 'Cliente_20']}}}, '1 a 2': {'Iteración_5': {'eps': 0.05, 'num_clusters': 2, 'clusters': {0: ['Cliente_1', 'Cliente_2', 'Cliente_3', 'Cliente_4', 'Cliente_6', 'Cliente_7', 'Cliente_8', 'Cliente_10', 'Cliente_14', 'Cliente_15', 'Cliente_16', 'Cliente_17', 'Cliente_18', 'Cliente_19'], 1: ['Cliente_5', 'Cliente_9', 'Cliente_11', 'Cliente_12', 'Cliente_13', 'Cliente_20']}}}, '1 a 3': {'Iteración_5': {'eps': 0.05, 'num_clusters': 2, 'clusters': {0: ['Cliente_1', 'Cliente_2', 'Cliente_3', 'Cliente_4', 'Cliente_6', 'Cliente_7', 'Cliente_8', 'Cliente_10', 'Cliente_14', 'Cliente_15', 'Cliente_16', 'Cliente_17', 'Cliente_18', 'Cliente_19'], 1: ['Cliente_5', 

{'1': {'Iteración_48': {'eps': 0.48000000000000026,
   'num_clusters': 1,
   'clusters': {0: ['Cliente_1',
     'Cliente_2',
     'Cliente_3',
     'Cliente_4',
     'Cliente_5',
     'Cliente_6',
     'Cliente_7',
     'Cliente_8',
     'Cliente_9',
     'Cliente_10',
     'Cliente_11',
     'Cliente_12',
     'Cliente_13',
     'Cliente_14',
     'Cliente_15',
     'Cliente_16',
     'Cliente_17',
     'Cliente_18',
     'Cliente_19',
     'Cliente_20']}}},
 '1 a 2': {'Iteración_5': {'eps': 0.05,
   'num_clusters': 2,
   'clusters': {0: ['Cliente_1',
     'Cliente_2',
     'Cliente_3',
     'Cliente_4',
     'Cliente_6',
     'Cliente_7',
     'Cliente_8',
     'Cliente_10',
     'Cliente_14',
     'Cliente_15',
     'Cliente_16',
     'Cliente_17',
     'Cliente_18',
     'Cliente_19'],
    1: ['Cliente_5',
     'Cliente_9',
     'Cliente_11',
     'Cliente_12',
     'Cliente_13',
     'Cliente_20']}}},
 '1 a 3': {'Iteración_5': {'eps': 0.05,
   'num_clusters': 2,
   'clusters': {0:

In [29]:
def coste_distancia(km, coste_vehiculo_km, total_coste):
    total_coste = total_coste +(km * coste_vehiculo_km)
    return total_coste

def total_duracion():
    pass


In [30]:
#FIN PRUEBAS
def rutas(df_loaction):
    rutas = [] #IMPORTANTE MAMAHUEVO
    for i in range(len(df_location) - 1):
        nodo_origen = df_location.loc[i, "Nodo"]
        nodo_destino = df_location.loc[i + 1, "Nodo"]
        ruta = nx.shortest_path(G, nodo_origen, nodo_destino, weight="length")
        rutas.append(ruta)

    ruta_completa = []
    for ruta in rutas:
        if ruta_completa:
            ruta_completa += ruta[1:]
        else:
            ruta_completa = ruta
    return ruta

ox.config(log_console=True, use_cache=True)

#ruta_completa = ruta_a_b + ruta_b_c[1:]  # Evitar duplicar el nodo intermedio
ruta_completa = []
ruta_completa = rutas(df_location)
print("Ruta completa (lista de nodos):", ruta_completa)

fig, ax = ox.plot_graph_route(G, ruta_completa, route_linewidth=4, node_size=0, bgcolor="white", route_color="blue")

C:\Users\JWinn01\AppData\Local\Temp\ipykernel_11464\1531108469.py:18: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(log_console=True, use_cache=True)


KeyError: 'Nodo'